In [ ]:
## and now q-vectors
## first, smooth everything pretty heavily:
temp_850 = ndimage.gaussian_filter(temp_850, sigma=10, order=0) * units('kelvin')
uwnd_850 = ndimage.gaussian_filter(uwnd_850, sigma=10, order=0) * units('m/s')
vwnd_850 = ndimage.gaussian_filter(vwnd_850, sigma=10, order=0) * units('m/s')

In [ ]:
# Compute the Q-vector components
uqvect, vqvect = mpcalc.q_vector(uwnd_850, vwnd_850, temp_850, 850*units.hPa, dx, dy)

# Compute the divergence of the Q-vectors calculated above
q_div = -2*mpcalc.divergence(uqvect, vqvect, dx, dy, dim_order='yx')


In [ ]:
q_div = ndimage.gaussian_filter(q_div, sigma=10, order=0) * units('meter/(kilogram second)')
q_div.units

In [ ]:
#Map Creation

# Set up Coordinate System for Plot and Transforms
dproj = ds.variables['LambertConformal_Projection']
globe = ccrs.Globe(ellipse='sphere', semimajor_axis=dproj.earth_radius,
                   semiminor_axis=dproj.earth_radius)
datacrs = ccrs.LambertConformal(central_latitude=dproj.latitude_of_projection_origin,
                                central_longitude=dproj.longitude_of_central_meridian,
                                standard_parallels=[dproj.standard_parallel],
                                globe=globe)
plotcrs = ccrs.LambertConformal(central_latitude=45., central_longitude=-100.,
                                standard_parallels=[30, 60])

fig = plt.figure(1, figsize=(14., 12))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, .02], bottom=.07, top=.99,
                       hspace=0.01, wspace=0.01)
ax = plt.subplot(gs[0], projection=plotcrs)

# Plot Titles
plt.title('850-hPa Geo. Heights (m), Temp (C),'
          ' Q-Vectors (m$^2$ kg$^{-1}$ s$^{-1}$) and Q-Vec Div. (*10$^{18}$ m s$^{-1}$ kg$^{-1}$)', loc='left')
plt.title('VALID: {}'.format(vtime[0]), loc='right')

# Plot Background
ax.set_extent([235., 290., 20., 58.], ccrs.PlateCarree())
ax.coastlines('50m', edgecolor='black', linewidth=0.75)
ax.add_feature(cfeature.STATES, linewidth=.5)

# Plot Height Contours
clev850 = np.arange(900, 3000, 30)
cs = ax.contour(lon, lat, hght_850.m, clev850, colors='black', linewidths=1.0,
                linestyles='solid', transform=ccrs.PlateCarree())
plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

# Plot temperature Contours
clevtemp850 = np.arange(-30, 20, 2)
cs2 = ax.contour(lon, lat, temp_850.to(units('degC')), clevtemp850, colors='grey',
                 linewidths=1.25, linestyles='dashed', transform=ccrs.PlateCarree())
plt.clabel(cs2, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

# Plot 850-hPa Q-Vector Divergence and scale
clevs_qdiv = list(range(-30, -4, 5))+list(range(5, 31, 5))
cf = ax.contourf(lon, lat, q_div*1e18, clevs_qdiv, extend='both',
                 cmap='bwr', transform=ccrs.PlateCarree())
cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True,
                  ticks=clevs_qdiv)
cb.set_label('Q-Vector Div. (*10$^{18}$ m s$^{-1}$ kg$^{-1}$)')

# Plot 850-hPa Q-vectors, scale to get nice sized arrows. Skip every 20th point when plotting.
ax.quiver(lon[::20,::20],lat[::20,::20],uqvect[::20,::20].m,vqvect[::20,::20].m,
          pivot='mid', color='black',
          scale=1e-11, scale_units='inches',
          transform=ccrs.PlateCarree())
